# Image Cow Recognition - Geolocation Calculation

## Imports

In [4]:
import numpy as np

## Read in files

In [5]:
file_name = "./cow_count_output.txt"

image_info = []
with open(file_name) as f:
    line = f.readline()
    while line != "":
        image_info.append(line)
        line = f.readline()

image_info = np.array(image_info)

## Conversion

In [6]:
TARGET_FILES = "./relative_final_result.txt"

### Image coordinate to Relative Coordiante

In [7]:
# Image pixel coordinate to relative coordinate
def image_to_relative(row, col):
    """
    image_to_relative
        parameters:
        - row: image row
        - col: image col
        method:
        - convert image coordinate to relative coordinate delta L and delta D        
    """
    row_int = int(row)
    col_int = int(col)
    delta_l = 2.14 + 5.32 * (10 ** -3) * np.abs(col_int - 6080 / 2) + -3.39 * (10 ** -4) * row_int + 4.66 * (10 ** -7) * (col_int - 6080 / 2)**2 + -2.14 * (10 ** -6) * np.abs(col_int - 6080/2) * row_int
    delta_d = 412303.05 * row_int ** (-1.33)
    return delta_l, delta_d

### Camera Information

In [8]:
camera_dictionary = {
    "TW2": {
        "C1_S": (3.23088, 84),
        "C2_S": (3.59664, 87),
        "C3_S": (3.32232, 80),
        "C3_N": (3.32232, 80),
        "C4_N": (3.29184, 84),
        "C5_N": (3.6576, 71)
    },
    "TW4": {
        "C1_SW": (3.41376, 88),
        "C2_S": (3.3528, 81),
        "C3_NE": (3.6576, 83),
        "C4_N": (3.62712, 83),
        "C5_NW": (3.71856, 82),
    }
}

camera_coordinates = {
    "TW1": (33.8798, 85.701),
    "TW2": (33.8875, 85.6942),
    "TW3": (34.3825, 85.6427),
    "TW4": (34.3823, 85.6447)
}

In [9]:
def convert_rc_to_xy(lat, long, gamma, r, c, direction):
    """
    Convert pixel coordinates to geographic coordinates
        parameters
        - lat: latitude of the camera
        - long: longitude of the camera
        - gamma: given gamma angle
        - r: image row
        - c: image column
        - direction: direction the camera is pointing at
        method:
        - convert image coordinate into geolocation
    """
    angle = np.deg2rad(45)
    
    # Getting the rotational matrix for differnet angle
    weight_class = {
        "N": np.array([[-1, 0], [0, -1]]),
        "E": np.array([[0, 1], [-1, 0]]),
        "W": np.array([[0, -1], [1, 0]]),
        "S": np.array([[1, 0], [0, 1]]),
        "NW": np.array([
                        [np.cos(angle), -np.sin(angle)], 
                        [np.sin(angle), np.cos(angle)]
                        ])  @ np.array([[-1, 0], [0, -1]]),
        "NE": np.array([
                        [np.cos(angle), -np.sin(angle)], 
                        [np.sin(angle), np.cos(angle)]
                        ])  @ np.array([[0, 1], [-1, 0]]),
        "SW": np.array([
                        [np.cos(angle), -np.sin(angle)], 
                        [np.sin(angle), np.cos(angle)]
                        ])  @ np.array([[0, -1], [1, 0]]),
        "SE": np.array([
                        [np.cos(angle), -np.sin(angle)], 
                        [np.sin(angle), np.cos(angle)]
                        ])  @ np.array([[1, 0], [0, 1]]),
                        
    }

    # Formula given by Chongya
    gamma = np.radians(gamma)
    cosgamma = np.cos(gamma)
    singamma = np.sin(gamma)
    dl, dd = image_to_relative(r,c)
    dx = dl*cosgamma - dd*singamma
    dy = dl*singamma + dd*cosgamma
    R=6378137

    dLat = dy/R
    dLon = dx/(R*np.cos(np.pi*lat/180))
    dy = dLat * 180/np.pi
    dx = dLon * 180/np.pi

    d_mat = weight_class[direction]
    dxp, dyp = d_mat @ np.array([[dx], [dy]])
    y = lat + dyp
    x = long + dxp

    return y, x

## Conversion function

In [15]:
def convert(file_name):
    with open(file_name, "w") as f:
        cameras = ["TW1", "TW2", "TW3", "TW4"]
        for infos in image_info:
            infos = infos.split(",")
            # strip the line info
            for i in range(0, len(infos)):
                infos[i] = infos[i].strip()
            # get the filename
            file_name = infos[0]
            cow_number = infos[1]

            # if now cow, simply write and continue
            if cow_number == '0':
                content = "%s, %s\n" % (file_name, cow_number)
                f.write(content)
            rest = infos[2:]
            coordinate = ""
            # get each coordinates
            for i in range(0, len(rest), 2):
                # get the row
                row = rest[i][1:]
                # get the column
                col = rest[i + 1][:-1]

                camera_direction = None

                # get the camera type and direction
                for camera in cameras:
                    if camera in file_name:
                        x_0, y_0 = camera_coordinates[camera]
                        camera_directions = camera_dictionary[camera]
                        camera_directions_keys = list(camera_directions)
                        for direction in camera_directions_keys:
                            if direction in file_name:
                                gamma = camera_dictionary[camera][direction][1]
                                compass_direction = ["NE", "SW", "SE", "NW", "E", "W", "S", "N"]
                                for d in compass_direction:
                                    if d in direction:
                                        camera_direction = d
                                        break
                                break
                        break
                # coordinate conversion
                geo_x, geo_y = convert_rc_to_xy(x_0, y_0, gamma, row, col, camera_direction)
                coordinate += "(%f; %f), " % (geo_x, geo_y)
                coordinate = coordinate[:-2]
                content = "%s, %s, %s\n" % (file_name, cow_number, coordinate)
            f.write(content)

### Convert to text and csv files

In [16]:
convert(TARGET_FILES)
convert(f"{TARGET_FILES[:-3]}csv")